<a href="https://colab.research.google.com/github/ny147/Stock_tax_generate/blob/main/Stock_Tax_Csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 538.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 58.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import fitz,re, glob, os, pdfplumber
from google.colab import drive
import os.path
from os import path

In [ ]:
def is_password_protected_pdf(pdf_file_path):
    doc = fitz.Document(pdf_file_path)
    if doc.needs_pass:
        return True
    return False

In [ ]:
def is_pdf_text_encrypted(pdf_file_path):
    doc = fitz.Document(pdf_file_path)
    if doc.metadata["encryption"] is not None:
        return True
    return False

In [ ]:
def decrypt_pdf(doc_dir,pdf_file_path, password,name):
    file_name = []
    doc = fitz.Document(pdf_file_path)
    if doc.authenticate(password):
        file_name = f"{name}_decrypted.pdf"
        doc.save(f"{doc_dir}/Decrypted/{file_name}")
        print("\Successfully decrypted PDF")
    else:
        print("\t Password incorrect!! Cannot decrypt PDF!!!")
    return f"{doc_dir}/Decrypted/{file_name}"

In [ ]:
def CONVERT_TO_PDF(path,name):
  pdf = pdfplumber.open(path)
  page = pdf.pages[0]
  text = page.extract_text_simple()
  pdf.close()

  df_raw = text.split('\n')
  index = 0
  for stindex in range(len(df_raw)-1,-1,-1):
    if re.match("^CCY",df_raw[stindex]):
      index = stindex
      break
  df_raw_extract_cc = df_raw[index::][1:-1]

  column = [ 'date','type','stock','unit','price per unit (USD)','currency','total price (USD)','Fee Include vat (USD)','Withhodling Tax (USD)','all price (USD)','total price (THB)',
  'Fee Include vat (THB)','Withhodling Tax (THB)', 'all price (THB)',]

  ls = []

  for i in range(0,len(df_raw_extract_cc),2):
    row = df_raw_extract_cc[i].split(" ")[1:] + df_raw_extract_cc[i+1].split(" ")[1:]
    ls.append(row)


  result_df = pd.DataFrame(ls,columns= column)

  return result_df

In [ ]:
def GENERATECSV(filelist):
  result = None
  for i,name in enumerate(filelist):
    df_temp = CONVERT_TO_PDF(name,f"number {i}")
    if result is None:
      result = df_temp

    else :
      result = pd.concat([result,df_temp])
  result.to_csv(f"{doc_dir}/result/summary.csv", index=False)

## SET UP FILE

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_dir = '/content/drive/MyDrive'

## setup directory
myFolder = "Tax"
password="enter your password"

doc_dir = f"{drive_dir}/{myFolder}"


## Create Drive

In [ ]:
if path.exists(f"{doc_dir}/result") == False:
  os.mkdir(f"{doc_dir}/result")
elif path.exists(f"{doc_dir}/Decrypted") == False:
  os.mkdir(f"{doc_dir}/Decrypted")

In [ ]:
fileList=[]
os.chdir(doc_dir)
for file in glob.glob("*.pdf"):
    filenamepath=f"{doc_dir}/{file}"
    if is_password_protected_pdf(filenamepath) == True:
      name = file.split(".")[0]
      print(filenamepath)
      fileList.append(decrypt_pdf(doc_dir,filenamepath,password,name))
    else :
      fileList.append(filenamepath)
    print(file)


/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_24062024105754.pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_24062024105754.pdf
/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_14062024102538.pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_14062024102538.pdf
/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_24052024104953 (1).pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_24052024104953 (1).pdf
/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_28032024102615.pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_28032024102615.pdf
/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_07032024102942.pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_07032024102942.pdf
/content/drive/MyDrive/Tax/Dime_offshore_confirmationNote_9680_01032024100432.pdf
\Successfully decrypted PDF
Dime_offshore_confirmationNote_9680_01032024100432.pdf
/c

In [ ]:
GENERATECSV(fileList)